### 1. Import modules

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pyodbc
import pandas as pd
import numpy as np

### 2. Import data to Pandas dataframe (Kidos_Kind_Relatie)

In [2]:
# Import pickles
df_zuigelingen = pd.read_pickle('4_Data\Pickles\zuigelingen_observaties.pkl')

In [3]:
# Set to True or False depending on whether you want to import new data
NEWIMPORT = False

if NEWIMPORT:
    # Replace server name if source changes
    server = 'AZ-ODB0\ODBWB'
    database = 'OnderzoeksDB_WB'

    # Create a connection string
    connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database}'

    # Establish connection
    connection = pyodbc.connect(connection_string)

    # Define columns to select
    selected_columns = ['Clientnummer',
                        'Relatienummer',
                        'Relatieband',
                        'Ouder',
                        'Geboortejaarmaand_kind',
                        'Geboortejaarmaand_relatie',
                        'Geboorteland_relatie',
                        'Geslacht_relatie']

    # Define a list of Clientnummers to include
    included_clientnummers = df_zuigelingen['Clientnummer'].unique()
    print(len(included_clientnummers))

    # Create SQL query string with specific columns and inclusion criteria
    query = f'SELECT {', '.join(selected_columns)} FROM Kidos_Kind_Relatie WHERE Clientnummer IN ({', '.join(included_clientnummers)})'

    # Execute the query and fetch the data into a Pandas DataFrame
    df_relaties_og = pd.read_sql(query, connection)

    # Close the connection
    connection.close()

    # Save df_relaties_og as a pickle
    df_relaties_og.to_pickle('4_Data\Pickles\\relaties_zuigelingen_og.pkl')
else:
    # Load the pickle
    df_relaties_og = pd.read_pickle('4_Data\Pickles\\relaties_zuigelingen_og.pkl')

### 3. Process

In [4]:
print(f'The amount of relations in the df_relaties is: {len(df_zuigelingen)}, the amount of unique client ID\'s is: {len(df_zuigelingen['Clientnummer'].unique())}')

The amount of relations in the df_relaties is: 5384, the amount of unique client ID's is: 5384


In [5]:
# Store copy of unique records of df_og in df
df_relaties = df_relaties_og.drop_duplicates()

# Set yearmonth's to NaN if it is '00'
df_relaties['Geboortejaarmaand_relatie'] = df_relaties['Geboortejaarmaand_relatie'].replace('00', np.NaN)

In [6]:
print(f'The amount of relations in the df_relaties is: {len(df_relaties)}, the amount of unique client ID\'s is: {len(df_relaties['Clientnummer'].unique())}')

The amount of relations in the df_relaties is: 18249, the amount of unique client ID's is: 5760


#### 3.1 Transform table structure - Siblings

In [7]:
# Create a pivot table with 'Clientnummer' as the index and 'Relatieband' as the columns
clients_frequencies = df_relaties.pivot_table(index='Clientnummer', columns='Relatieband', aggfunc='size', fill_value=0)

# Select only the 'Broer of zus' column
clients_relations_siblings = clients_frequencies['Broer of zus'].reset_index(name='Siblings')

# Show head
print(clients_relations_siblings.shape)
clients_relations_siblings.head()

(5760, 2)


,Clientnummer,Siblings
0,10074469,4
1,10080706,2
2,8161527,1
3,8163825,1
4,8163828,2


#### 3.2 Transform table structure - Parent1

In [8]:
df_relaties['Relatieband'].unique()

array(['Eigen (biologische) vader van het kind', 'Broer of zus',
       'Eigen (biologische) moeder van het kind', 'Anders',
       'Partner/vriend van de vader of moeder (stiefvader)',
       'Adoptief vader', 'Halfbroer of halfzus', 'Adoptief moeder',
       'Partner/vriendin van de vader of moeder (stiefmoeder)',
       'Pleegmoeder', 'Pleegvader'], dtype=object)

In [9]:
df_relaties['Ouder'].unique()

array(['Ouder2', None, 'Ouder1'], dtype=object)

In [10]:
# Get all unique combinations of Relatieband and Ouder and their value counts
df_relaties['Relatieband_Ouder'] = df_relaties['Relatieband'] + ' - ' + df_relaties['Ouder']
df_relaties['Relatieband_Ouder'].value_counts()

Relatieband_Ouder
Eigen (biologische) moeder van het kind - Ouder1                  4147
Eigen (biologische) vader van het kind - Ouder2                   3951
Anders - Ouder2                                                    894
Anders - Ouder1                                                    808
Eigen (biologische) moeder van het kind - Ouder2                   807
Eigen (biologische) vader van het kind - Ouder1                    781
Partner/vriend van de vader of moeder (stiefvader) - Ouder2         12
Partner/vriendin van de vader of moeder (stiefmoeder) - Ouder2       9
Adoptief moeder - Ouder2                                             7
Partner/vriend van de vader of moeder (stiefvader) - Ouder1          4
Adoptief vader - Ouder2                                              3
Partner/vriendin van de vader of moeder (stiefmoeder) - Ouder1       3
Adoptief moeder - Ouder1                                             2
Adoptief vader - Ouder1                                    

In [11]:
# Make list of parent types
parents = ['Eigen (biologische) vader van het kind',
           'Eigen (biologische) moeder van het kind',
           'Partner/vriendin van de vader of moeder (stiefmoeder)',
           'Adoptief vader',
           'Partner/vriend van de vader of moeder (stiefvader)',
           'Adoptief moeder',
           'Pleegvader',
           'Pleegmoeder']

# For every unique 'Clientnummer' in df_relaties, count the amount of rows where 'Relatieband' is in parents or 'Ouder' is either Ouder1 or Ouder2
df_relaties['Parents'] = df_relaties['Relatieband'].isin(parents) | df_relaties['Ouder'].isin(['Ouder1', 'Ouder2'])
clients_relations_parents = df_relaties.groupby('Clientnummer')['Parents'].sum().reset_index(name='Parents')

In [12]:
clients_relations_parents[clients_relations_parents['Parents'] > 2]

,Clientnummer,Parents
132,8188545,3
535,8251856,3
1011,8310255,4
1466,8373787,3
1509,8377670,3
1697,8402602,3
2045,8459843,3
2046,8461448,4
2121,8481356,3
2780,8586421,3


In [13]:
# Remove all rows where Relatieband is not in relevant_relaties
df_relaties_parent1 = df_relaties[df_relaties['Ouder'] == 'Ouder1']

# Pivot from long to wide format
clients_relations_parent1 = (df_relaties_parent1.pivot_table(index='Clientnummer',
                                                    columns='Ouder',
                                                    values=['Geboortejaarmaand_relatie', 'Geboorteland_relatie', 'Geslacht_relatie'],
                                                    aggfunc='first')
                                                    .pipe(lambda d: d.set_axis(d.columns.map(lambda x: f'{x[0]}_ouder1'), axis=1))
                                                    .reset_index())

# Show head
print(clients_relations_parent1.shape)
clients_relations_parent1.head()

(5747, 4)


,Clientnummer,Geboortejaarmaand_relatie_ouder1,Geboorteland_relatie_ouder1,Geslacht_relatie_ouder1
0,10074469,199503,Syrië,onbekend
1,10080706,199501,Syrië,vrouwelijk
2,8161527,197905,Nederland,mannelijk
3,8163825,198906,Nederland,vrouwelijk
4,8163828,199002,Nederland,vrouwelijk


In [14]:
clients_relations_parent1.isna().mean()

Clientnummer                        0.0
Geboortejaarmaand_relatie_ouder1    0.0
Geboorteland_relatie_ouder1         0.0
Geslacht_relatie_ouder1             0.0
dtype: float64

#### 3.3 Transform table structure - Parent2

In [15]:
# Remove all rows where Relatieband is not in relevant_relaties
df_relaties_parent2 = df_relaties[df_relaties['Ouder'] == 'Ouder2']

# Pivot from long to wide format
clients_relations_parent2 = (df_relaties_parent2.pivot_table(index='Clientnummer',
                                                    columns='Ouder',
                                                    values=['Geboortejaarmaand_relatie', 'Geboorteland_relatie', 'Geslacht_relatie'],
                                                    aggfunc='first')
                                                    .pipe(lambda d: d.set_axis(d.columns.map(lambda x: f'{x[0]}_ouder2'), axis=1))
                                                    .reset_index())

# Show head
print(clients_relations_parent2.shape)
clients_relations_parent2.head()

(5683, 4)


,Clientnummer,Geboortejaarmaand_relatie_ouder2,Geboorteland_relatie_ouder2,Geslacht_relatie_ouder2
0,10074469,198501,Syrië,onbekend
1,10080706,199108,Syrië,mannelijk
2,8161527,198012,Nederland,vrouwelijk
3,8163825,198701,Nederland,mannelijk
4,8163828,198911,Nederland,mannelijk


In [16]:
clients_relations_parent2.isna().mean()

Clientnummer                        0.000000
Geboortejaarmaand_relatie_ouder2    0.000352
Geboorteland_relatie_ouder2         0.000176
Geslacht_relatie_ouder2             0.000000
dtype: float64

#### 3.4 Merge siblings and parents

In [17]:
# Merge clients_siblings_frequencies and clients_relations on 'Clientnummer' keeping all rows from clients_siblings_frequencies
clients_relations = (clients_relations_siblings
                     .merge(clients_relations_parent1, on='Clientnummer', how='left')
                     .merge(clients_relations_parent2, on='Clientnummer', how='left'))

# Show head
print(clients_relations.shape)
clients_relations.head()

(5760, 8)


,Clientnummer,Siblings,Geboortejaarmaand_relatie_ouder1,Geboorteland_relatie_ouder1,Geslacht_relatie_ouder1,Geboortejaarmaand_relatie_ouder2,Geboorteland_relatie_ouder2,Geslacht_relatie_ouder2
0,10074469,4,199503,Syrië,onbekend,198501,Syrië,onbekend
1,10080706,2,199501,Syrië,vrouwelijk,199108,Syrië,mannelijk
2,8161527,1,197905,Nederland,mannelijk,198012,Nederland,vrouwelijk
3,8163825,1,198906,Nederland,vrouwelijk,198701,Nederland,mannelijk
4,8163828,2,199002,Nederland,vrouwelijk,198911,Nederland,mannelijk


#### 3.5 Align fathers with parent2 and mothers with parent1

In [18]:
# Replace all NaN with 'onbekend'
clients_relations = clients_relations.fillna('onbekend')

# If parent1 is masculine and parent2 is feminine or unknown, switch them
mask = ((clients_relations['Geslacht_relatie_ouder1'] == 'mannelijk') &
        ((clients_relations['Geslacht_relatie_ouder2'] == 'vrouwelijk') | (clients_relations['Geslacht_relatie_ouder2'] == 'onbekend')) |
        ((clients_relations['Geslacht_relatie_ouder2'] == 'vrouwelijk') & (clients_relations['Geslacht_relatie_ouder1'] == 'onbekend')))

# Switch
clients_relations.loc[mask, ['Geboortejaarmaand_relatie_ouder1',
                             'Geboorteland_relatie_ouder1',
                             'Geslacht_relatie_ouder1',
                             'Geboortejaarmaand_relatie_ouder2',
                             'Geboorteland_relatie_ouder2',
                             'Geslacht_relatie_ouder2']] = clients_relations.loc[mask, ['Geboortejaarmaand_relatie_ouder2',
                                                                                        'Geboorteland_relatie_ouder2',
                                                                                        'Geslacht_relatie_ouder2',
                                                                                        'Geboortejaarmaand_relatie_ouder1',
                                                                                        'Geboorteland_relatie_ouder1',
                                                                                        'Geslacht_relatie_ouder1']].values

In [19]:
# Show head
print(clients_relations.shape)
clients_relations.head()

(5760, 8)


,Clientnummer,Siblings,Geboortejaarmaand_relatie_ouder1,Geboorteland_relatie_ouder1,Geslacht_relatie_ouder1,Geboortejaarmaand_relatie_ouder2,Geboorteland_relatie_ouder2,Geslacht_relatie_ouder2
0,10074469,4,199503,Syrië,onbekend,198501,Syrië,onbekend
1,10080706,2,199501,Syrië,vrouwelijk,199108,Syrië,mannelijk
2,8161527,1,198012,Nederland,vrouwelijk,197905,Nederland,mannelijk
3,8163825,1,198906,Nederland,vrouwelijk,198701,Nederland,mannelijk
4,8163828,2,199002,Nederland,vrouwelijk,198911,Nederland,mannelijk


### 4. Transform monthyear dates

In [20]:
# Change the data type of the column to string
clients_relations['Geboortejaarmaand_relatie_ouder1'] = clients_relations['Geboortejaarmaand_relatie_ouder1'].astype(str)
clients_relations['Geboortejaarmaand_relatie_ouder2'] = clients_relations['Geboortejaarmaand_relatie_ouder2'].astype(str)

# Replace "onbekend" with np.nan
clients_relations['Geboortejaarmaand_relatie_ouder1'] = clients_relations['Geboortejaarmaand_relatie_ouder1'].replace('onbekend', np.nan)
clients_relations['Geboortejaarmaand_relatie_ouder2'] = clients_relations['Geboortejaarmaand_relatie_ouder2'].replace('onbekend', np.nan)

# Change month dtypes
clients_relations['Geboortejaarmaand_relatie_ouder1'] = pd.to_datetime(clients_relations['Geboortejaarmaand_relatie_ouder1'], format='%Y%m').dt.to_period('M').dt.strftime('%Y-%m')
clients_relations['Geboortejaarmaand_relatie_ouder2'] = pd.to_datetime(clients_relations['Geboortejaarmaand_relatie_ouder2'], format='%Y%m').dt.to_period('M').dt.strftime('%Y-%m')

In [21]:
clients_relations.isna().mean()

Clientnummer                        0.000000
Siblings                            0.000000
Geboortejaarmaand_relatie_ouder1    0.000521
Geboorteland_relatie_ouder1         0.000000
Geslacht_relatie_ouder1             0.000000
Geboortejaarmaand_relatie_ouder2    0.015451
Geboorteland_relatie_ouder2         0.000000
Geslacht_relatie_ouder2             0.000000
dtype: float64

In [22]:
clients_relations[clients_relations['Clientnummer'] == '8850643']

,Clientnummer,Siblings,Geboortejaarmaand_relatie_ouder1,Geboorteland_relatie_ouder1,Geslacht_relatie_ouder1,Geboortejaarmaand_relatie_ouder2,Geboorteland_relatie_ouder2,Geslacht_relatie_ouder2
4332,8850643,2,1986-02,Nederland,vrouwelijk,1982-03,Nederland,mannelijk


### 5. Save as .pkl

In [23]:
# Save df_observaties as .pkl file
clients_relations.to_pickle('4_Data/Pickles/zuigelingen_relaties.pkl')